# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\triangleq \sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

6. Extreme Learning Machines: 
    Just like the Bayesian network but then last layer is trained on the training set using KRidge!

#### Mode:
Software/Hardware Testing or Real-Deal?

In [25]:
trial_run = True

### Simulation Method:

In [26]:
# Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
## Real-world data version
f_unknown_mode = "Extreme_Learning_Machine"
### Dataset Option 1
dataset_option = 'SnP'
### Dataset Option 2
# dataset_option = 'crypto'
Depth_Bayesian_DNN = 1
N_Random_Features = 10**1
## Simulated Data version
# f_unknown_mode = "DNN_with_Random_Weights"
width = 10

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.1

# GD with Randomized Input
# f_unknown_mode = "GD_with_randomized_input"
# GD_epochs = 50

# SDE with fractional Driver
# f_unknown_mode = "Rough_SDE"
N_Euler_Steps = 10**2
Hurst_Exponent = 0.75

# f_unknown_mode = "Rough_SDE_Vanilla"
## Define Process' dynamics in (2) cell(s) below.

## Problem Dimension

In [27]:
problem_dim = 1
if f_unknown_mode != 'Extreme_Learning_Machine':
    width = int(2*(problem_dim+1))

#### Vanilla fractional SDE:
If f_unknown_mode == "Rough_SDE_Vanilla" is selected, then we can specify the process's dynamics.  

In [28]:
#--------------------------#
# Define Process' Dynamics #
#--------------------------#
drift_constant = 0.1
volatility_constant = 0.01

# Define DNN Applier
def f_unknown_drift_vanilla(x):
    x_internal = x
    x_internal = drift_constant*x_internal
    return x_internal
def f_unknown_vol_vanilla(x):
    x_internal = volatility_constant*np.diag(np.ones(problem_dim))
    return x_internal

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [29]:
train_test_ratio = .2
N_train_size = 10

Monte-Carlo Paramters

In [30]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**2

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [31]:
# Hyper-parameters of Cover
delta = 0.1
Proportion_per_cluster = .75

## Dependencies and Auxiliary Script(s)

In [32]:
# %run Loader.ipynb
exec(open('Loader.py').read())
# Load Packages/Modules
exec(open('Init_Dump.py').read())
import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready


# Simulate or Parse Data

In [33]:
# %run Data_Simulator_and_Parser.ipynb
exec(open('Data_Simulator_and_Parser.py').read())

---------------------------------------
Beginning Data-Parsing/Simulation Phase
---------------------------------------
Deciding on Which Simulator/Parser To Load
Setting/Defining: Internal Parameters
Deciding on Which Type of Data to Get/Simulate


 16%|█▌        | 16/100 [00:00<00:00, 153.92it/s]

#================================================#
 Training Datasize: 479 and test datasize: 120.  
#================================================#
Simulating Output Data for given input data


100%|██████████| 100/100 [00:00<00:00, 171.30it/s]

----------------------------------
Done Data-Parsing/Simulation Phase
----------------------------------


#### Scale Data
This is especially important to avoid exploding gradient problems when training the ML-models.

In [34]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Run Main:

In [35]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
# %run Universal_Measure_Valued_Networks_Backend.ipynb
exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

------------------------------
Running script for main model!
------------------------------


100%|██████████| 359/359 [00:00<00:00, 4217.29it/s]

Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    5.1s finished


Epoch 1/2
30/30 [==============================] - 0s 2ms/step - loss: 5.9128 - accuracy: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 0s 1ms/step
Training Classifer Portion of Type Model: Done!


...

#--------------------#
 Get Training Error(s)
#--------------------#


  3%|▎         | 4/120 [00:00<00:03, 31.72it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 120/120 [00:03<00:00, 30.15it/s]

#------------------------#
 Get Testing Error(s): END
#------------------------#
                                        DNM     MC-Oracle
W1-95L                             0.007710  0.000000e+00
W1                                 0.008373  0.000000e+00
W1-95R                             0.008952  0.000000e+00
M-95L                              0.010786  0.000000e+00
M                                  0.011770  0.000000e+00
M-95R                              0.012559  0.000000e+00
N_Par                          18909.000000  0.000000e+00
Train_Time                        21.544275  1.620220e+09
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00
------------------------------------
Done: Running script for main model!
------------------------------------


---
# Run: All Benchmarks

## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [36]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


100%|██████████| 120/120 [00:00<00:00, 4281.65it/s]

---------------------
Training: ENET - Done
---------------------
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                        DNM     MC-Oracle         ENET
W1-95L                             0.008006  0.000000e+00            -
W1                                 0.008373  0.000000e+00            -
W1-95R                             0.008952  0.000000e+00            -
M-95L                              0.010786  0.000000e+00   0.00726812
M                                  0.011770  0.000000e+00   0.00770548
M-95R                              0.012559  0.000000e+00   0.00816749
N_Par                          18909.000000  0.000000e+00          958
Train_Time                        21.544275  1.620220e+09  1.62022e+09
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0840s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.4s finished
100%|██████████| 120/120 [00:00<00:00, 4530.10it/s]

#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame


...

#------------#
 Get Error(s) 
#------------#


  0%|          | 0/120 [00:00<?, ?it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 120/120 [00:00<00:00, 4350.19it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.00726812   
M                                  0.011770  0.000000e+00   0.00770548   
M-95R                              0.012559  0.000000e+00   0.00816749   
N_Par                          18909.000000  0.000000e+00          958   
Train_Time                        21.544275  1.620220e+09  1.62022e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914406   

                                  KRidge        GBRF        DNN  
W1-95L                                 -           -          -  
W1                                     -         

# Summary of Point-Mass Regression Models

#### Training Model Facts

In [37]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.00726812   
M                                  0.011770  0.000000e+00   0.00770548   
M-95R                              0.012559  0.000000e+00   0.00816749   
N_Par                          18909.000000  0.000000e+00          958   
Train_Time                        21.544275  1.620220e+09  1.62022e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914406   

                                  KRidge        GBRF        DNN  
W1-95L                                 -           -          -  
W1                                     -           -          -  
W1-95R                                 -           -         

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,0.008006,0.000000e+00,-,-,-,-
W1,0.008373,0.000000e+00,-,-,-,-
W1-95R,0.008952,0.000000e+00,-,-,-,-
M-95L,0.010786,0.000000e+00,0.00726812,0.0486667,0.00210335,0.0281057
M,0.011770,0.000000e+00,0.00770548,0.0766662,0.00257861,0.0317936
M-95R,0.012559,0.000000e+00,0.00816749,0.105086,0.00305558,0.0378587
N_Par,18909.000000,0.000000e+00,958,0,470690,651
Train_Time,21.544275,1.620220e+09,1.62022e+09,0.762621,3.90242,5.11038
Test_Time/MC-Oracle_Test_Time,0.666456,1.000000e+00,0.000914406,0.0961755,0.0403798,0.58296


#### Testing Model Facts

In [38]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                                        DNM     MC-Oracle          ENET  \
W1-95L                             0.007710  0.000000e+00  7.633688e-03   
W1                                 0.008373  0.000000e+00  8.463183e-03   
W1-95R                             0.008952  0.000000e+00  9.630304e-03   
M-95L                              0.010786  0.000000e+00  5.672209e-03   
M                                  0.011770  0.000000e+00  6.462542e-03   
M-95R                              0.012559  0.000000e+00  7.293723e-03   
N_Par                          18909.000000  0.000000e+00  9.580000e+02   
Train_Time                        21.544275  1.620220e+09  1.620220e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  9.144058e-04   

                                 KRidge           GBRF         DNN  
W1-95L                         0.020189       0.007268    0.008595  
W1                             0.042885       0.008421    0.010016  
W1-95R                         0.081862   

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,0.007710,0.000000e+00,7.633688e-03,0.020189,0.007268,0.008595
W1,0.008373,0.000000e+00,8.463183e-03,0.042885,0.008421,0.010016
W1-95R,0.008952,0.000000e+00,9.630304e-03,0.081862,0.009251,0.011057
M-95L,0.010786,0.000000e+00,5.672209e-03,0.048667,0.002103,0.028106
M,0.011770,0.000000e+00,6.462542e-03,0.076666,0.002579,0.031794
M-95R,0.012559,0.000000e+00,7.293723e-03,0.105086,0.003056,0.037859
N_Par,18909.000000,0.000000e+00,9.580000e+02,0.000000,470690.000000,651.000000
Train_Time,21.544275,1.620220e+09,1.620220e+09,0.762621,3.902419,5.110375
Test_Time/MC-Oracle_Test_Time,0.666456,1.000000e+00,9.144058e-04,0.096176,0.040380,0.582960


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\Sigma}(x)\hat{\Sigma}^{\top})\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to 
(2\pi)^{-\frac{d}{2}}\det(\hat{\Sigma}(x))^{-\frac{1}{2}} \, e^{ -\frac{1}{2}(\cdot - \hat{\mu}(x))^{{{\!\mathsf{T}}}} \hat{\Sigma}(x)^{-1}(\cdot - \hat{\mu}(x)) } \mu \in \mathcal{G}_d\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [39]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.8s remaining:    1.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.1s finished
100%|██████████| 479/479 [00:00<00:00, 4402.18it/s]

Infering Parameters for Deep Gaussian Network to train on!
Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.1s finished


Epoch 1/2
30/30 [==============================] - 0s 1ms/step - loss: 0.4826 - mse: 0.4003 - mae: 0.4826 - mape: 709.4232
Epoch 2/2
8/8 [==============================] - 0s 830us/step


...

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


  0%|          | 0/120 [00:00<?, ?it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------------------#
 Get Testing Errors for: Gaussian Models
#--------------------------------------#


100%|██████████| 120/120 [00:00<00:00, 2522.31it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.00726812   
M                                  0.011770  0.000000e+00   0.00770548   
M-95R                              0.012559  0.000000e+00   0.00816749   
N_Par                          18909.000000  0.000000e+00          958   
Train_Time                        21.544275  1.620220e+09  1.62022e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914406   

                         

In [40]:
print("Prediction Quality (Updated): Test")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Prediction Quality (Updated): Test
                                        DNM     MC-Oracle          ENET  \
W1-95L                             0.007710  0.000000e+00  7.633688e-03   
W1                                 0.008373  0.000000e+00  8.463183e-03   
W1-95R                             0.008952  0.000000e+00  9.630304e-03   
M-95L                              0.010786  0.000000e+00  5.672209e-03   
M                                  0.011770  0.000000e+00  6.462542e-03   
M-95R                              0.012559  0.000000e+00  7.293723e-03   
N_Par                          18909.000000  0.000000e+00  9.580000e+02   
Train_Time                        21.544275  1.620220e+09  1.620220e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  9.144058e-04   

                                 KRidge           GBRF         DNN       GPR  \
W1-95L                         0.020189       0.007268    0.008595  0.026742   
W1                             0.042885       0.008421

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,0.007710,0.000000e+00,7.633688e-03,0.020189,0.007268,0.008595,0.026742,0.905528
W1,0.008373,0.000000e+00,8.463183e-03,0.042885,0.008421,0.010016,0.047363,0.930014
W1-95R,0.008952,0.000000e+00,9.630304e-03,0.081862,0.009251,0.011057,0.064547,0.950431
M-95L,0.010786,0.000000e+00,5.672209e-03,0.048667,0.002103,0.028106,0.045379,0.063727
M,0.011770,0.000000e+00,6.462542e-03,0.076666,0.002579,0.031794,0.057781,0.068120
M-95R,0.012559,0.000000e+00,7.293723e-03,0.105086,0.003056,0.037859,0.069837,0.070762
N_Par,18909.000000,0.000000e+00,9.580000e+02,0.000000,470690.000000,651.000000,0.000000,651.000000
Train_Time,21.544275,1.620220e+09,1.620220e+09,0.762621,3.902419,5.110375,6.620785,4.255152
Test_Time/MC-Oracle_Test_Time,0.666456,1.000000e+00,9.144058e-04,0.096176,0.040380,0.582960,0.222001,0.565207


In [41]:
print("Prediction Quality (Updated): Train")
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

Prediction Quality (Updated): Train
                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.00726812   
M                                  0.011770  0.000000e+00   0.00770548   
M-95R                              0.012559  0.000000e+00   0.00816749   
N_Par                          18909.000000  0.000000e+00          958   
Train_Time                        21.544275  1.620220e+09  1.62022e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914406   

                                  KRidge        GBRF        DNN       GPR  \
W1-95L                                 -           -          -  0.026742   
W1                                     -           -          -  0.04

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,0.008006,0.000000e+00,-,-,-,-,0.026742,0.930756
W1,0.008373,0.000000e+00,-,-,-,-,0.047363,0.941348
W1-95R,0.008952,0.000000e+00,-,-,-,-,0.064547,0.954946
M-95L,0.010786,0.000000e+00,0.00726812,0.0486667,0.00210335,0.0281057,0.045379,0.071870
M,0.011770,0.000000e+00,0.00770548,0.0766662,0.00257861,0.0317936,0.057781,0.072942
M-95R,0.012559,0.000000e+00,0.00816749,0.105086,0.00305558,0.0378587,0.069837,0.074750
N_Par,18909.000000,0.000000e+00,958,0,470690,651,0.000000,651.000000
Train_Time,21.544275,1.620220e+09,1.62022e+09,0.762621,3.90242,5.11038,6.620785,4.255152
Test_Time/MC-Oracle_Test_Time,0.666456,1.000000e+00,0.000914406,0.0961755,0.0403798,0.58296,0.222001,0.565207


# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [42]:
if output_dim == 1:
    # %run Mixture_Density_Network.ipynb
    exec(open('Mixture_Density_Network.py').read())

  0%|          | 0/479 [00:00<?, ?it/s]

Preparing Training Outputs for MDNs using EM-Algorithm


100%|██████████| 479/479 [00:58<00:00,  8.14it/s]

Prepared Training Outputs for MDNs using EM-Algorithm!
Deep Feature Builder - Ready
(0)
Training Mixture Density Network (MDN): Means: Start!
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.1s finished


...

Training Mixture Density Network (MDN): Mixture Coefficients: END!
#--------------------#
 Get Training Error(s)
#--------------------#


 16%|█▌        | 19/120 [00:00<00:00, 181.74it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------#
 Get Test Error(s)
#--------------------#


100%|██████████| 120/120 [00:00<00:00, 183.54it/s]

#---------------------#
 Get Test Error(s): END
#---------------------#
#---------------------------#
 Get Training Error(s): Begin
#---------------------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------#
 Get Testing Error(s): Begin
#--------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
Updated DataFrame
-------------------------------------------------
Train
                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.0072

## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

---
# Final Results
---

## Prasing Quality Metric Results

#### Finalizing Saving
**Note:** *We do it in two steps since the grid sometimes does not want to write nicely...*

In [43]:
## Write Performance Metrics
### Incase caption breaks
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 float_format="{:0.3g}".format)
text_file = open((results_tables_path+"/Final_Results/"+"ZZZ_CAPTION_Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS___CAPTION.tex"), "w")
text_file.write("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+".")
text_file.close()


### Incase caption does not break
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 caption=("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+"."),
                                 float_format="{:0.3g}".format)

# For Terminal Runner(s):

In [44]:
# For Terminal Running
print("===================")
print("Predictive Quality:")
print("===================")
print(Summary_pred_Qual_models)
print("===================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")
Summary_pred_Qual_models

Predictive Quality:
                                        DNM     MC-Oracle         ENET  \
W1-95L                             0.008006  0.000000e+00            -   
W1                                 0.008373  0.000000e+00            -   
W1-95R                             0.008952  0.000000e+00            -   
M-95L                              0.010786  0.000000e+00   0.00726812   
M                                  0.011770  0.000000e+00   0.00770548   
M-95R                              0.012559  0.000000e+00   0.00816749   
N_Par                          18909.000000  0.000000e+00          958   
Train_Time                        21.544275  1.620220e+09  1.62022e+09   
Test_Time/MC-Oracle_Test_Time      0.666456  1.000000e+00  0.000914406   

                                  KRidge        GBRF        DNN       GPR  \
W1-95L                                 -           -          -  0.026742   
W1                                     -           -          -  0.047363   
W1-95R  

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN,MDN
W1-95L,0.008006,0.000000e+00,-,-,-,-,0.026742,0.930756,1.004274
W1,0.008373,0.000000e+00,-,-,-,-,0.047363,0.941348,1.020559
W1-95R,0.008952,0.000000e+00,-,-,-,-,0.064547,0.954946,1.049264
M-95L,0.010786,0.000000e+00,0.00726812,0.0486667,0.00210335,0.0281057,0.045379,0.071870,0.069465
M,0.011770,0.000000e+00,0.00770548,0.0766662,0.00257861,0.0317936,0.057781,0.072942,0.072350
M-95R,0.012559,0.000000e+00,0.00816749,0.105086,0.00305558,0.0378587,0.069837,0.074750,0.076754
N_Par,18909.000000,0.000000e+00,958,0,470690,651,0.000000,651.000000,16947.000000
Train_Time,21.544275,1.620220e+09,1.62022e+09,0.762621,3.90242,5.11038,6.620785,4.255152,0.174989
Test_Time/MC-Oracle_Test_Time,0.666456,1.000000e+00,0.000914406,0.0961755,0.0403798,0.58296,0.222001,0.565207,754.183479


---
# Fin
---

---